In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="-1"

In [8]:
import pixellib
from pixellib.instance import instance_segmentation
import cv2

segment_video = instance_segmentation()
segment_video.load_model("mask_rcnn_coco.h5")

capture = cv2.VideoCapture("accident1.mp4")
#capture = cv2.VideoCapture("accident2.mp4")
b=0
while (capture.isOpened()):
    ret, frame = capture.read()
    if not ret:
        break
        
    frame=frame[:frame.shape[0]-20,:,:]
    result = segment_video.segmentFrame(frame, show_bboxes=True)
    
    for i in range(len(result[0]['rois'])):
                
        if result[0]['class_ids'][i] == 3 :   # car  
            x1 = result[0]['rois'][i][1]
            y1 = result[0]['rois'][i][0]
            x2 = result[0]['rois'][i][3]
            y2 = result[0]['rois'][i][2]

            mid_x = (x1/frame.shape[0]+x2/frame.shape[0])/2
            mid_y = (y1/frame.shape[1]+y2/frame.shape[1])/2

            apx_distance = round(((1 - (y2/frame.shape[1] - y1/frame.shape[1]))**3), 1)

            #cv2.putText(frame, '{}'.format(apx_distance), (int((x1+x2)/2),int((y1+y2)/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

            if apx_distance <=0.6:
                if mid_y > 0.2 :
                    cv2.putText(frame, 'danger', (int((x1+x2)/2),int((y1+y2)/2)), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,255), 3)                    
                    b=1

    if(b):
        with open("../distance_between_cars_output.txt", "w") as file: 
            file.write("danger")
            b=0
    else:
        with open("../distance_between_cars_output.txt", "w") as file: 
                        file.write("safe")
                
    cv2.imshow("frame", frame)
    
    if cv2.waitKey(25) & 0xff == ord('q'):
          break
    
cv2.destroyAllWindows()
capture.release()